In [40]:
import numpy as np
import urllib
from sklearn import preprocessing
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from numpy import loadtxt, where
from pylab import scatter, show, legend, xlabel, ylabel
from matplotlib.colors import ListedColormap
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [41]:
#Preprocessed clean version of input file
input_file = "Cleaned_file.csv"

In [42]:
df = pd.read_csv(input_file)

In [43]:
# Printing Data Shape
df.shape

(11670, 6)

In [44]:
df.head()

,District,Date,Primary Type,Arrest,Crime Weightage,Label
0,11,5/3/16 21:00,NARCOTICS,True,5,1
1,8,5/3/16 21:35,THEFT,False,1,1
2,9,5/3/16 14:45,ASSAULT,False,3,0
3,8,5/3/16 18:42,BATTERY,False,2,1
4,9,5/3/16 19:30,CRIMINAL DAMAGE,False,3,1


In [45]:
#dataset = np.loadtxt(input_file, delimiter=",")

In [46]:
# Giving numeric values for text data
primaryType = preprocessing.LabelEncoder()
arrest = preprocessing.LabelEncoder()
#date = preprocessing.LabelEncoder()

In [47]:
df["Primary Type"] = primaryType.fit_transform(df["Primary Type"])
df["Arrest"] = arrest.fit_transform(df["Arrest"])
#df["Date"] = date.fit_transform(df["Date"])

In [48]:
df['Date'] = df['Date'].apply(lambda x: x.split('/')[0])

In [49]:
#df['Date'] = df['Date'].dt.month

In [50]:
df.head()

,District,Date,Primary Type,Arrest,Crime Weightage,Label
0,11,5,16,1,5,1
1,8,5,28,0,1,1
2,9,5,1,0,3,0
3,8,5,2,0,2,1
4,9,5,6,0,3,1


In [51]:
#df["Date"] = date.inverse_transform(df["Date"])

In [52]:
#df.head()

In [53]:
array = df.values

In [54]:
#columns = ["District", "Date", "Primary Type", "Arrest", "Crime Weightage"]

In [55]:
# Features
X = list(array[:,0:5])

In [56]:
# Target
Y = list(array[:,5])

In [57]:
totalTarget = df["Label"].count()

In [58]:
# Count of 0s in the target
zeroCount = df[(df["Label"] == 0)].count()[0]

In [59]:
# Count of 1s in the target
oneCount = df[(df["Label"] == 1)].count()[0]

In [60]:
#Probability of 0s in the target
probZero = zeroCount / totalTarget

In [61]:
#Probability of 1s in the target
probOne = oneCount / totalTarget

In [62]:
print("Probability of 0s in the Target ", '%0.3f'%(probZero))

Probability of 0s in the Target  0.588


In [63]:
# Baseline for positive in the Target 
print("Probability of 1s in the Target ", '%0.3f'%(probOne))

Probability of 1s in the Target  0.412


In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.70, random_state=0)

In [65]:
# Calculating accuracy of LOGISTIC REGRESSION using CROSS_VAL_SCORE
C= [5,2,1,0.1,0.01,0.001,0.002,0.0001,0.00001]
for c in C: 
    clf = LogisticRegression(C = c, max_iter = 1000, random_state=0, solver='sag')
    scoresLR = cross_val_score(clf,X_train,y_train,cv=10,scoring='accuracy')
    print("Accuracy with C = ",c,": %0.5f(+/- %0.5f)" %(scoresLR.mean(), scoresLR.std()*2))

### So above we can see that Logistic Regression accuracy is best when C >= 1.0

In [67]:
# Calculating accuracy of DECISION TREE CLASSIFIER using CROSS_VAL_SCORE
dtree = tree.DecisionTreeClassifier()
scoresDT = cross_val_score(dtree,X_train,y_train,cv=10,scoring='accuracy')
print("Accuracy with Decision Tree: %0.5f (+/- %0.5f)" % (scoresDT.mean(), scoresDT.std() * 2))

Accuracy : 0.99951 (+/- 0.00162)


In [68]:
# Calculating accuracy of NAIVE BAYES using CROSS_VAL_SCORE
gnb = GaussianNB()
scores = cross_val_score(gnb,X_train,y_train,cv=10,scoring='accuracy')
print("Accuracy with Gaussian Naive Bayes: %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.63920 (+/- 0.03629)


In [83]:
L = [0.001,0.002,0.003,0.0001,0.00001, 0.000001]
for l in L: 
    MLPC = MLPClassifier(random_state=0, learning_rate='adaptive',max_iter=10000, learning_rate_init=l)
    scoresMLPC = cross_val_score(MLPC,X_train,y_train,cv=10,scoring='accuracy')
    print("Accuracy with MLPC Classifier with",l,": %0.5f (+/- %0.5f)" % (scoresMLPC.mean(), scoresMLPC.std() * 2))

Accuracy with MLPC Classifier with 0.001 : 0.82578 (+/- 0.03658)
Accuracy with MLPC Classifier with 0.002 : 0.83987 (+/- 0.03972)
Accuracy with MLPC Classifier with 0.003 : 0.85161 (+/- 0.05520)
Accuracy with MLPC Classifier with 0.0001 : 0.80828 (+/- 0.02259)
Accuracy with MLPC Classifier with 1e-05 : 0.66834 (+/- 0.03039)
Accuracy with MLPC Classifier with 1e-06 : 0.58435 (+/- 0.01264)


In [84]:
svclf = SVC(cache_size=1000) # Increased Cache size for larger datasets
scores = cross_val_score(svclf,X_train,y_train,cv=10,scoring='accuracy')
print("Accuracy with Support Vector Machines: %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))

Accuracy with Support Vector Machines: 0.96523 (+/- 0.01620)


In [71]:
'''Y_pred    = clf.predict(X_test)
Y_score   = clf.decision_function(X_test)
accuracy  = metrics.accuracy_score(y_test, Y_pred)
precision = metrics.precision_score(y_test, Y_pred)
recall    = metrics.recall_score(y_test, Y_pred)
AUC       = metrics.roc_auc_score(y_test, Y_score)'''

'Y_pred    = clf.predict(X_test)\nY_score   = clf.decision_function(X_test)\naccuracy  = metrics.accuracy_score(y_test, Y_pred)\nprecision = metrics.precision_score(y_test, Y_pred)\nrecall    = metrics.recall_score(y_test, Y_pred)\nAUC       = metrics.roc_auc_score(y_test, Y_score)'

In [72]:
#print('%0.3f'%accuracy,'%0.3f'%precision, '%0.3f'%recall,'%0.3f'%AUC)

In [73]:
#scores = cross_val_score(clf,X_train,y_train,cv=10,scoring='accuracy')

In [74]:
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [78]:
#scores = cross_val_score(clf,X_train,y_train,cv=10,scoring='accuracy')